In [ ]:
!pip install google-cloud-language

In [ ]:
import pandas as pd
from google.cloud import language_v2
from google.cloud import language_v1
from google.oauth2 import service_account

SERVICE_ACCOUNT_FILE = 'service_account.json'

SCOPES = ['https://www.googleapis.com/auth/cloud-language']

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE
)
credentials=credentials.with_scopes(SCOPES)

In [ ]:
import time
def analyze_bluesky_sentiment_v2_csv(csv_file, output_file, text_row):

    df = pd.read_csv(csv_file, encoding='utf-8')


    client = language_v2.LanguageServiceClient(credentials=credentials)


    results = []

    for _, row in df.iterrows():
        post = row[text_row]


        document = {
            "content": post,
            "type": language_v2.Document.Type.PLAIN_TEXT,
            "language_code": "pt"
        }

        try:

            response = client.analyze_sentiment(request={"document": document})
            sentiment = response.document_sentiment


            if 0.25 < sentiment.score <= 1:
                sentiment_api = "positivo"
            elif -1 <= sentiment.score < -0.25:
                sentiment_api = "negativo"
            elif -0.25 <= sentiment.score <= 0.25:
                if sentiment.magnitude > 1:
                    sentiment_api = "misto"
                else:
                    sentiment_api = "neutro"
            else:
                sentiment_api = "undefined"  # Se algum valor não for contemplado


            results.append({
                "uri": row["uri"],
                "record.$type": row["record.$type"],
                "record.createdAt": row["record.createdAt"],
                "replyCount.$numberLong": row["replyCount.$numberLong"],
                "repostCount.$numberLong": row["repostCount.$numberLong"],
                "likeCount.$numberLong": row["likeCount.$numberLong"],
                "quoteCount.$numberLong": row["quoteCount.$numberLong"],
                "score": sentiment.score,
                "magnitude": sentiment.magnitude,
                "sentiment_api": sentiment_api,
                "record.text": post
            })

        except Exception as e:
            print(f"Erro ao processar: {e}")
            continue


        time.sleep(0.1)  # Pausa de 100ms por requisição (10 requisições por segundo)


    result_df = pd.DataFrame(results)


    result_df.to_csv(output_file, index=False)

Análise de sentimentos posts relacionados ao **STF**

In [ ]:
columns = [
    "uri",
    "record.$type",
    "record.createdAt",
    "replyCount.$numberLong",
    "repostCount.$numberLong",
    "likeCount.$numberLong",
    "quoteCount.$numberLong",
    "score",
    "magnitude",
    "sentiment_api",
    "record.text"
]

empty_df = pd.DataFrame(columns=columns)

empty_df.to_csv('analise_sentimento_stf.csv', index=False)

In [ ]:
analyze_bluesky_sentiment_v2_csv('stf_posts_filtered.csv', 'analise_sentimento_stf.csv', 'record.text')

Análise de sentimentos posts relacionados ao **Bluesky**

In [ ]:
columns = [
    "uri",
    "record.$type",
    "record.createdAt",
    "replyCount.$numberLong",
    "repostCount.$numberLong",
    "likeCount.$numberLong",
    "quoteCount.$numberLong",
    "score",
    "magnitude",
    "sentiment_api",
    "record.text"
]

empty_df = pd.DataFrame(columns=columns)

empty_df.to_csv('analise_sentimento_bluesky.csv', index=False)

In [ ]:
analyze_bluesky_sentiment_v2_csv('bluesky_posts_filtered.csv', 'analise_sentimento_bluesky.csv', 'record.text')

Análise de sentimentos posts relacionados ao **X** / **Twitter**

In [ ]:
columns = [
    "uri",
    "record.$type",
    "record.createdAt",
    "replyCount.$numberLong",
    "repostCount.$numberLong",
    "likeCount.$numberLong",
    "quoteCount.$numberLong",
    "score",
    "magnitude",
    "sentiment_api",
    "record.text"
]

empty_df = pd.DataFrame(columns=columns)

empty_df.to_csv('analise_sentimento_twitter.csv', index=False)

In [ ]:
analyze_bluesky_sentiment_v2_csv('twitter_x_posts_filtered.csv', 'analise_sentimento_twitter.csv', 'record.text')